# AWS Strands SDK MCP for PTO and Weather


AWS Strands SDK
Amazon Bedrock
FastMCP

In [3]:
# Import required libraries
import os
from dotenv import load_dotenv
from strands import Agent
from strands.models.bedrock import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.sse import sse_client

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


## Verify AWS Credentials

Before creating the model, let's verify your AWS credentials are valid:


In [2]:
# Check AWS credentials
import boto3
from datetime import datetime

try:
    # Create a Bedrock client to test credentials
    bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')
    
    # Try to get caller identity to verify credentials
    sts = boto3.client('sts', region_name='us-west-2')
    identity = sts.get_caller_identity()
    
    print("✓ AWS Credentials are valid!")
    print(f"  Account: {identity['Account']}")
    print(f"  User ARN: {identity['Arn']}")
    
    # If using temporary credentials, check expiration
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials.token:  # Temporary credentials have a token
        print("\n⚠️  You are using temporary credentials.")
        print("   If they expire, you'll need to refresh them.")
    
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("\n🔧 To fix this:")
    print("   1. Run: aws configure")
    print("   2. Or update your .env file with new AWS credentials")
    print("   3. Or get fresh credentials from AWS SSO/Academy")
    print("\n   Then restart the kernel and run from the beginning.")


✓ AWS Credentials are valid!
  Account: 977099023120
  User ARN: arn:aws:sts::977099023120:assumed-role/AWSReservedSSO_AdministratorAccess_8133186339cd89a9/chuongho@myhofamily.net

⚠️  You are using temporary credentials.
   If they expire, you'll need to refresh them.


In [3]:
# Initialize the Amazon Bedrock model
try:
    # Using Claude 3.5 Sonnet V2 - widely available and works reliably
    model = BedrockModel(
        model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
        boto_session=session
    )
    print("✓ Bedrock model initialized successfully!")
    print("  Model: Claude 3.5 Sonnet V2")
    print("  Model ID: us.anthropic.claude-3-5-sonnet-20241022-v2:0")
    print("  Region: us-west-2")
except Exception as e:
    print(f"⚠️  Error initializing Bedrock model: {e}")


✓ Bedrock model initialized successfully!
  Model: Claude 3.5 Sonnet V2
  Model ID: us.anthropic.claude-3-5-sonnet-20241022-v2:0
  Region: us-west-2


In [4]:
# Define the system prompt for the PTO agent
system_prompt = """You are a helpful HR assistant that helps employees check their PTO (Paid Time Off) hours. 
Be friendly, concise, and professional. Use the available MCP tools to provide accurate information about PTO balances."""

# Create MCP client connection
pto_mcp_client = MCPClient(lambda: sse_client("http://127.0.0.1:8000/sse"))
weather_mcp_client = MCPClient(lambda: sse_client("http://127.0.0.1:8001/sse"))


print("✓ MCP client created - use within context manager for agent operations")


✓ MCP client created - use within context manager for agent operations


In [5]:
# TEST 1: Check available PTO hours (within MCP context manager)
query = "How many PTO hours do I have available?"
print(f"User: {query}\n")

with pto_mcp_client, weather_mcp_client:
    pto_agent = Agent(
        model=model,
        system_prompt=system_prompt,
        tools = pto_mcp_client.list_tools_sync() + weather_mcp_client.list_tools_sync()
    )
    response = pto_agent(query)
    print(f"Agent: {response}\n")



User: How many PTO hours do I have available?

I'll help you check your available PTO hours.
Tool #1: get_pto_hours
You have 94 PTO hours available. Let me know if you'd like to know how many full days this translates to or if you have any other questions about your PTO!Agent: You have 94 PTO hours available. Let me know if you'd like to know how many full days this translates to or if you have any other questions about your PTO!




In [ ]:
# TEST 2: Check weather in San Diego
query = "What is the weather like in 92128?"
print(f"User: {query}\n")

with pto_mcp_client, weather_mcp_client:
    pto_agent = Agent(
        model=model,
        system_prompt=system_prompt,
        tools = pto_mcp_client.list_tools_sync() + weather_mcp_client.list_tools_sync()
    )
    response = pto_agent(query)
    print(f"Agent: {response}\n")



## 🌐 Gradio Web Interface

Let's create an interactive web interface for the PTO agent using Gradio:


In [6]:
# Import Gradio and createe gradio wrapper function
import gradio as gr

def pto_agent_chat(user_query):
    """
    Wrapper function for the PTO agent to work with Gradio
    Takes a user query and returns the agent's response
    """
    try:
        with pto_mcp_client, weather_mcp_client:
            tools = pto_mcp_client.list_tools_sync() + weather_mcp_client.list_tools_sync()
            pto_agent = Agent(
            model=model,
            system_prompt=system_prompt,
            tools = pto_mcp_client.list_tools_sync() + weather_mcp_client.list_tools_sync()
        )
            response = pto_agent(user_query)
            return str(response)
    except Exception as e:
        return f"❌ Error: {str(e)}\n\nPlease make sure the MCP server is running at http://127.0.0.1:8000/sse"

print("✓ Gradio wrapper function created")


/Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Day 2/genai-immersion-day-300-demos/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Gradio wrapper function created


In [7]:
# Create Gradio interface for PTO Agent
with gr.Blocks(title="Chuong's Super Cool PTO and Weather Agent", theme=gr.themes.Soft()) as pto_demo:
    gr.Markdown("# 🏖️ Chuong's Super Cool PTO and Weather Agent")
    gr.Markdown("### Powered by Claude 3.5 Sonnet V2, Strands SDK, FastMCP on Amazon Bedrock")
    
    # Display current PTO balance
    gr.Markdown(f"""
    **Connected to MCP Server at http://127.0.0.1:8000/sse**
    
    Ask me anything about your PTO! Or ask me about the weather!
    - Check your available hours and days
    - See if you can take time off
    - Plan your vacation time
    """)
    
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(
                label="💬 Ask about your PTO", 
                lines=3,
                placeholder="e.g., How many PTO hours do I have?"
            )
            submit_btn = gr.Button("🚀 Ask Agent", variant="primary", size="lg")
        
    with gr.Row():
        agent_output = gr.Textbox(
            label="🤖 Agent Response", 
            lines=10,
            show_copy_button=True
        )
    
    # Example questions
    gr.Examples(
        examples=[
            "How many PTO hours do I have available?",
            "How many days of PTO can I take?",
            "Can I take a full day off next week?",
            "Do I have enough PTO for a half day?",
            "What's my PTO balance in days?"
        ],
        inputs=user_input,
        label="📝 Example Questions (click to try)"
    )
    
    submit_btn.click(
        fn=pto_agent_chat,
        inputs=user_input,
        outputs=agent_output
    )
    
    user_input.submit(
        fn=pto_agent_chat,
        inputs=user_input,
        outputs=agent_output
    )

print("✓ Gradio interface created")

pto_demo.launch(share=True)

✓ Gradio interface created
* Running on local URL:  http://127.0.0.1:7860
I'll help you check your PTO balance.
Tool #1: get_pto_days
You have 11.75 days of PTO available, which equals 94 hours. Let me know if you'd like to know anything else about your PTO balance!
Could not create share link. Missing file: /Users/chuongho/.cache/huggingface/gradio/frpc/frpc_darwin_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.3
3. Move the file to this location: /Users/chuongho/.cache/huggingface/gradio/frpc


I notice you're asking about the weather in Alexandria, but I'll need the specific ZIP code to check the weather there. Could you please provide the ZIP code for the area in Alexandria you're interested in? Once you share that, I can look up the current weather conditions for you.I'm an HR assistant primarily focused on helping with PTO (Paid Time Off) related questions. While I can check weather information, I'll need a specific ZIP code for San Francisco to provide that information. Could you please provide the ZIP code you'd like to check? For example, one of San Francisco's ZIP codes is 94102.

If you have any questions about PTO balances, time off availability, or related HR matters, I'd be happy to help with those!I'll check the current weather conditions for zip code 94102.
Tool #1: get_weather_by_zipcode
Currently in San Francisco (94102):
- Temperature: 62.1°F (16.7°C)
- Condition: Partly cloudy
- Humidity: 75%
- Wind: 5.4 mph
- Feels like: 62.1°F

Is there anything specific a

In [ ]:
# Close the Gradio interface when done
pto_demo.close()
